## 2.9 当列表不是首选时

比如，要存放1000 万个浮点数的话，数组（array）的效率要高得多，因为数组在背后存的并不是float 对象，而是数字的机器翻译，也就是字节表述。

### 2.9.1 数组

如果我们需要一个只包含数字的列表，那么 array.array 比 list 更高效。  
数组支持所有跟可变序列有关的操作，包括 .pop、.insert 和 .extend。  
另外，数组还提供从文件读取和存入文件的更快的方法，如 .frombytes 和 .tofile。  

In [10]:
from array import array
import random

In [11]:
random.seed(123)
floats = array('d', (random.random() for i in range(10**6)))
print(floats[-1])

0.2686141961467833


In [23]:
fp = open('floats.bin', 'wb') # wb二进制格式打开文件
floats.tofile(fp)

In [13]:
floats2 = array('d')  # 'd' : float
fp = open('floats.bin', 'rb')
floats2.fromfile(fp, 10**6)
fp.close()

In [14]:
print(floats2[-1])
print(floats == floats2)

0.2686141961467833
True


note : python open() 函数用于打开一个文件，创建一个 file 对象，相关的方法才可以调用它进行读写。  
open(name[, mode[, buffering]])  
name : 文件名称  
mode : 决定打开文件的模式：只读，写入，追加等。默认文件访问模式为只读(r)。  
buffering : 如果 buffering 的值被设为 0，就不会有寄存。如果 buffering 的值取 1，访问文件时会寄存行。如果将 buffering 的值设为大于 1 的整数，表明了这就是的寄存区的缓冲大小。如果取负值，寄存区的缓冲大小则为系统默认。  
+ wb : 以二进制格式打开一个文件只用于写入。如果该文件已存在则打开文件，并从开头开始编辑，即原有内容会被删除。如果该文件不存在，创建新文件。

从 Python 3.4 开始，数组类型不再支持诸如 list.sort() 这种就地排序方法。  
要给数组排序的话，得用 sorted 函数新建一个数组：

In [25]:
a = array('d', (random.random() for i in range(5)))
print(a)
a = array(a.typecode, sorted(a)) # a.typecode指定数组类型
print(a)

array('d', [0.7107466634580362, 0.7925700233781673, 0.056366803846116476, 0.8520626813356386, 0.3629228400778325])
array('d', [0.056366803846116476, 0.3629228400778325, 0.7107466634580362, 0.7925700233781673, 0.8520626813356386])


### 2.9.2 内存视图

memoryview 是一个内置类，它能让用户在不复制内容的情况下操作同一个数组的不同切片。  
memoryview.cast 的概念跟数组模块类似，能用不同的方式读写同一块内存数据，而且内容字节不会随意移动。

In [2]:
import array

note: array.array的第一个参数是type code，主要的作用是声明后面数组的类型  
'h'都表示signed short，'B'表示unsigned char  
https://docs.python.org/3/library/array.html

In [4]:
number= array.array('h', [-2, -1, 0, 1, 2])  # signed short
memv = memoryview(number)
len(memv)

5

In [6]:
memv[0]

-2

In [12]:
memv_oct = memv.cast('B')  # unsigned char
memv_oct.format

'B'

In [13]:
memv_oct.itemsize

1

以列表的形式查看 memv_oct 的内容。

In [9]:
memv_oct.tolist()

[254, 255, 255, 255, 0, 0, 1, 0, 2, 0]

note: 为什么memv_oct.tolist()产生的元素比原始数组多了一倍？
因为memv.cast(‘B’)把memv转换成一个unsigned char int的新memoryview，并返回给memv_oct。  
signed short int在内存中是以2个字节存储，而unsigned char int在内存中则是1个字节存储。  
signed short int类型的原码最高位表示正负，0代表正数，1代表负数。  
它们内存中是以补码的形式存储的，其中正数的补码和原码相同；负数的补码，是其原码除符号位（即最高位，也就是最后一位）外，其余全部取反，再加1。  
所以，signed short int类型的-2，其原码为0100 0000 0000 0010，除符号位取反，为1011 1111 1111 1111，再加1，为0111 1111 1111 1111。当以unsigned char int类型读出来的时候，就成了254 255了；-1亦是同理，即255 255。

### 2.9.3 NumPy和SciPy

对 numpy.ndarray 的行和列进行基本操作

In [26]:
import numpy
a = numpy.arange(12)
a

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [27]:
type(a)

numpy.ndarray

In [28]:
a.shape

(12,)

In [30]:
a.shape = 3, 4
a

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [31]:
a[2]

array([ 8,  9, 10, 11])

In [32]:
a[2, 1]

9

In [33]:
a[:, 1]

array([1, 5, 9])

In [34]:
a.transpose()

array([[ 0,  4,  8],
       [ 1,  5,  9],
       [ 2,  6, 10],
       [ 3,  7, 11]])

### 2.9.4 双向队列和其他形式的队列

利用 .append 和 .pop 方法，我们可以把列表当作栈或者队列来用  
但是列表由于删除第一个元素或者是在第一个元素前添加元素非常耗时，不适合作双向队列  
collections.deque 类（双向队列）是一个线程安全、可以快速从两端添加或者删除元素的数据类型  

In [60]:
from collections import deque
dq = deque(range(10), maxlen=10)
dq

deque([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [61]:
dq.rotate(3)
dq

deque([7, 8, 9, 0, 1, 2, 3, 4, 5, 6])

In [62]:
dq.rotate(-4)
dq

deque([1, 2, 3, 4, 5, 6, 7, 8, 9, 0])

In [63]:
dq.appendleft(-1)
dq

deque([-1, 1, 2, 3, 4, 5, 6, 7, 8, 9])

当试图对一个已满（len(d) == d.maxlen）的队列做尾部添加操作的时候，它头部的元素会被删除掉

In [64]:
dq.extend([11, 22, 33])
dq

deque([3, 4, 5, 6, 7, 8, 9, 11, 22, 33])

In [65]:
dq.extendleft([10, 20, 30, 40])
dq

deque([40, 30, 20, 10, 3, 4, 5, 6, 7, 8])

extendleft(iter) 方法会把迭代器里的元素逐个添加到双向队列的左边，因此迭代器里的元素会逆序出现在队列里。

In [66]:
dq.appendleft([22, 33, 44])
dq

deque([[22, 33, 44], 40, 30, 20, 10, 3, 4, 5, 6, 7])

概率题：彩票，从20个数里面选5个作为中奖号码，1个作为特殊号码，购买彩票的人会有5个号码，如果5个号码和中奖号码全部一致则为一等奖，如果有4个号码与中奖号码一致且剩下的一个数字与特殊号码一致则为二等奖，如果有4个号码与中奖号码一致且剩下的一个数字与特殊号码不一致则为三等奖，求一、二、三等奖的中奖概率。

note: 使用scipy计算排列组合的具体数值  
from scipy.special import comb, perm  
perm(n,m)   计算排列数A(n,m)  
comb(n,,)   计算组合数C(n,m)  

In [22]:
from scipy.special import comb, perm

一等奖：$$\frac{1}{C_{20}^5}$$

In [19]:
1 / comb(20,5) * 100

0.006449948400412796

二等奖：$$\frac{C_5^4\times 1}{C_{20}^5}$$

In [20]:
5 / comb(20,5) * 100

0.032249742002063986

三等奖：$$\frac{C_5^4\times C_{14}^1}{C_{20}^5}$$

In [21]:
5 * 14 / comb(20,5) * 100

0.45149638802889575